In [1]:
import pandas as pd
import json
from datasets import load_dataset

In [ ]:
data = load_dataset('swesynth/SWE-Synth', split='train').to_pandas()
traj = load_dataset('swesynth/SWE-Synth_Moatless-SFT-Trajectories', split='train').to_pandas()
variants_metadata = load_dataset('swesynth/SWE-Synth_Variants-Metadata', split='train').to_pandas()

- Total # variants
- Fix patches
- Lines edited
- Diff hunk
- Files edited
- Fail tests
- Test log length
- Steps per traj

In [9]:
print(f"Number of total variants: {len(data)}")
print(f"Number of total trajectories / fix patches: {len(traj)}")

Number of total variants: 9459
Number of total trajectories / fix patches: 3018


In [12]:
print("Number of variants per repo:")
data['repo'].value_counts().sort_index()

Number of variants per repo:


repo
bokeh/bokeh                623
conan-io/conan            1525
dask/dask                  691
facebookresearch/hydra    2525
getmoto/moto               222
iterative/dvc             1607
pydantic/pydantic         2266
Name: count, dtype: int64

In [14]:
data

,instance_id,repo,base_commit,version,setup_patch,patches,environment_setup_commit,FAIL_TO_PASS,PASS_TO_PASS,problem_statement,_metadata
0,getmoto_moto-339309c9af4188006d9592469d52193f5...,getmoto/moto,339309c9af4188006d9592469d52193f57249b1e,4.1,--- a/moto/s3/exceptions.py\n+++ b/moto/s3/exc...,None,339309c9af4188006d9592469d52193f57249b1e,"[""tests/test_s3/test_s3_acl.py::test_put_bucke...","[""tests/test_sagemaker/test_sagemaker_endpoint...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/moto/s3/exceptio...
1,getmoto_moto-339309c9af4188006d9592469d52193f5...,getmoto/moto,339309c9af4188006d9592469d52193f57249b1e,4.1,--- a/moto/ec2/responses/spot_instances.py\n++...,[diff --git a/moto/ec2/responses/spot_instance...,339309c9af4188006d9592469d52193f57249b1e,"[""tests/test_ec2/test_spot_instances.py::test_...","[""tests/test_batch_simple/test_batch_jobs.py::...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/moto/ec2/respons...
2,getmoto_moto-339309c9af4188006d9592469d52193f5...,getmoto/moto,339309c9af4188006d9592469d52193f57249b1e,4.1,--- a/moto/s3/exceptions.py\n+++ b/moto/s3/exc...,[diff --git a/moto/iam/access_control.py b/mot...,339309c9af4188006d9592469d52193f57249b1e,"[""tests/test_s3/test_s3_auth.py::test_load_une...","[""tests/test_sagemaker/test_sagemaker_endpoint...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/moto/s3/exceptio...
3,getmoto_moto-339309c9af4188006d9592469d52193f5...,getmoto/moto,339309c9af4188006d9592469d52193f57249b1e,4.1,--- a/moto/s3/utils.py\n+++ b/moto/s3/utils.py...,None,339309c9af4188006d9592469d52193f57249b1e,"[""tests/test_s3/test_s3.py::test_multiple_dele...","[""tests/test_sagemaker/test_sagemaker_endpoint...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/moto/s3/utils.py...
4,getmoto_moto-339309c9af4188006d9592469d52193f5...,getmoto/moto,339309c9af4188006d9592469d52193f57249b1e,4.1,--- a/moto/cognitoidp/responses.py\n+++ b/moto...,None,339309c9af4188006d9592469d52193f57249b1e,"[""tests/test_cognitoidp/test_cognitoidp.py::te...","[""tests/test_cognitoidp/test_cognitoidp.py::te...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/moto/cognitoidp/...
...,...,...,...,...,...,...,...,...,...,...,...
9454,conan-io_conan-715264206ab3f7b1b438586a4cd2bc4...,conan-io/conan,715264206ab3f7b1b438586a4cd2bc4f07c7599d,1.55,--- a/conans/client/cache/cache.py\n+++ b/cona...,None,715264206ab3f7b1b438586a4cd2bc4f07c7599d,"[""conans/test/integration/graph_lock/version_r...","[""conans/test/integration/build_requires/profi...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/conans/client/ca...
9455,conan-io_conan-715264206ab3f7b1b438586a4cd2bc4...,conan-io/conan,715264206ab3f7b1b438586a4cd2bc4f07c7599d,1.55,--- a/conans/client/rest/rest_client.py\n+++ b...,None,715264206ab3f7b1b438586a4cd2bc4f07c7599d,"[""conans/test/integration/command/remote_test....",[],=================================== FAILURES =...,{'reverse_patch_diff': '--- a/conans/client/re...
9456,conan-io_conan-0c1624d2dd3b0278c1cf6f66f8dcc7b...,conan-io/conan,0c1624d2dd3b0278c1cf6f66f8dcc7bd1aa9ec48,2.0,--- a/conan/tools/build/__init__.py\n+++ b/con...,None,0c1624d2dd3b0278c1cf6f66f8dcc7bd1aa9ec48,"[""conans/test/integration/toolchains/cmake/tes...","[""conans/test/functional/toolchains/cmake/test...",=================================== FAILURES =...,{'reverse_patch_diff': '--- a/conan/tools/buil...
9457,conan-io_conan-0c1624d2dd3b0278c1cf6f66f8dcc7b...,conan-io/conan,0c1624d2dd3b0278c1cf6f66f8dcc7bd1aa9ec48,2.0,--- a/conan/cli/commands/config.py\n+++ b/cona...,None,0c1624d2dd3b0278c1cf6f66f8dcc7bd1aa9ec48,"[""conans/test/integration/command/config_test....",[],=================================== FAILURES =...,{'reverse_patch_diff': '--- a/conan/cli/comman...


## Test cases

In [25]:
(
    data
    .assign(
        FAIL_TO_PASS=lambda x: x['FAIL_TO_PASS'].apply(json.loads),
        PASS_TO_PASS=lambda x: x['PASS_TO_PASS'].apply(json.loads),
        num_fail_to_pass=lambda x: x['FAIL_TO_PASS'].apply(len),
        num_total_test_case=lambda x: x['FAIL_TO_PASS'].apply(len) + x['PASS_TO_PASS'].apply(len),
    )
    .agg({
        'num_fail_to_pass': ['mean', 'median'],
        'num_total_test_case': ['mean', 'median'],
    })
)

,num_fail_to_pass,num_total_test_case
mean,107.531346,254.918279
median,8.000000,72.000000


In [27]:
(
    data
    .assign(
        FAIL_TO_PASS=lambda x: x['FAIL_TO_PASS'].apply(json.loads),
        PASS_TO_PASS=lambda x: x['PASS_TO_PASS'].apply(json.loads),
        num_fail_to_pass=lambda x: x['FAIL_TO_PASS'].apply(len),
        num_total_test_case=lambda x: x['FAIL_TO_PASS'].apply(len) + x['PASS_TO_PASS'].apply(len),
    )
    .groupby('repo')['num_fail_to_pass'].median().astype(int)
)

repo
bokeh/bokeh                3
conan-io/conan             8
dask/dask                 18
facebookresearch/hydra    13
getmoto/moto               3
iterative/dvc              6
pydantic/pydantic          9
Name: num_fail_to_pass, dtype: int64

## Patch analysis

In [ ]:
# Helper functions

import re
from unidiff import PatchSet

def count_diff_hunks(patch_diff):
    patch_set = PatchSet(patch_diff)
    hunk_count = sum(len(file) for file in patch_set)  # Each file contains hunks
    return hunk_count

def get_changed_files_from_diff(diff: str) -> set[str]:
    source_files = {
        patch_file.path
        for patch_file in PatchSet(diff)
    }
    return source_files

def count_edited_lines(patch: str) -> int:
    added_lines = 0
    removed_lines = 0
    
    for line in patch.splitlines():
        if line.startswith('+') and not line.startswith('+++'):
            added_lines += 1
        elif line.startswith('-') and not line.startswith('---'):
            removed_lines += 1
    
    total_changes = added_lines + removed_lines
    return total_changes


In [38]:
with pd.option_context('display.float_format', '{:.2f}'.format):
    display(
        data
        [['repo', 'patches']]
        .explode('patches').rename(columns={'patches': 'patch'}).dropna()
        .assign(
            num_edited_lines=lambda x: x['patch'].apply(count_edited_lines),
            num_diff_hunks=lambda x: x['patch'].apply(count_diff_hunks),
            num_changed_files=lambda x: x['patch'].apply(lambda x: len(get_changed_files_from_diff(x))),
        )
        .groupby('repo')
        .agg({
            'num_edited_lines': 'mean',
            'num_diff_hunks': 'mean',
            'num_changed_files': 'mean',
        })
    )

    print("Mean across all repos:")
    display(
        data
        [['repo', 'patches']]
        .explode('patches').rename(columns={'patches': 'patch'}).dropna()
        .assign(
            num_edited_lines=lambda x: x['patch'].apply(count_edited_lines),
            num_diff_hunks=lambda x: x['patch'].apply(count_diff_hunks),
            num_changed_files=lambda x: x['patch'].apply(lambda x: len(get_changed_files_from_diff(x))),
        )
        .agg({
            'num_edited_lines': 'mean',
            'num_diff_hunks': 'mean',
            'num_changed_files': 'mean',
        })
    )

,num_edited_lines,num_diff_hunks,num_changed_files
repo,,,
bokeh/bokeh,4.95,1.22,1.08
conan-io/conan,6.00,1.36,1.05
dask/dask,7.87,1.22,1.02
facebookresearch/hydra,5.11,1.18,1.04
getmoto/moto,5.68,1.31,1.04
iterative/dvc,6.34,1.25,1.06
pydantic/pydantic,5.88,1.27,1.09


Mean across all repos:


num_edited_lines    5.73
num_diff_hunks      1.25
num_changed_files   1.06
dtype: float64

## Number steps per trajectory

In [51]:
print("Avg number of steps per trajectory:")
traj['messages'].apply(len).mean().round(2)

Avg number of steps per trajectory:


8.68

In [55]:
with pd.option_context('display.float_format', '{:.2f}'.format):
    display(
        data
        .merge(traj, on='instance_id')
        .assign(traj_length=lambda x: x['messages'].apply(len))
        .groupby('repo')['traj_length'].mean()
    )

repo
bokeh/bokeh              9.34
conan-io/conan           8.32
dask/dask                7.43
facebookresearch/hydra   8.65
getmoto/moto             9.28
iterative/dvc            8.62
pydantic/pydantic        8.64
Name: traj_length, dtype: float64

## Test log length

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-32B-Instruct")
def count_tokens(text: str) -> int:
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return len(tokens)

In [11]:
from datasets import Dataset
def _map_fn(x):
    return {'log_length': count_tokens(x['problem_statement'])}

_t = Dataset.from_pandas(data).map(_map_fn, num_proc=40).to_pandas()

Map (num_proc=40):   1%|          | 103/9459 [01:17<56:01,  2.78 examples/s] Token indices sequence length is longer than the specified maximum sequence length for this model (3026518 > 32768). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (438629 > 32768). Running this sequence through the model will result in indexing errors
Map (num_proc=40): 100%|██████████| 9459/9459 [19:29<00:00,  8.09 examples/s]


In [16]:
# _t = (data.assign(log_length=lambda x: x['problem_statement'].progress_apply(count_tokens)))
display(
    _t
    .groupby('repo')['log_length'].median().astype(int).sort_index()
)
print(
    f"Median of log length across all repo: ",
    _t['log_length'].median().astype(int)
)

repo
bokeh/bokeh                4414
conan-io/conan             9361
dask/dask                 67398
facebookresearch/hydra    93632
getmoto/moto              14108
iterative/dvc             42632
pydantic/pydantic          3802
Name: log_length, dtype: int64

Median of log length across all repo:  18842


## Type of variants 

In [ ]:
num_test_coverage_based_variants = len(variants_metadata[variants_metadata['mutation_info'].apply(lambda y: y['strategy']) == 'PriorityAwareMutationStrategy'])
print(f"Number of test-coverage based selection variants: {num_test_coverage_based_variants} ({num_test_coverage_based_variants / len(variants_metadata):.2%})")
print(f"Number of random selection variants: {len(variants_metadata) - num_test_coverage_based_variants} ({(len(variants_metadata) - num_test_coverage_based_variants) / len(variants_metadata):.2%})")

Number of test-coverage based selection variants: 3867 (40.88%)
Number of random selection variants: 5592 (59.12%)
